In [7]:
coof = [0, 0, 0.3, 0.3, 0.05]

transactions_file_name = 'data/JULY1-15TRXDATA.xlsx'
relevancy_file_name = 'data/Data_RX.xlsx'
transactions_to_get_reward = 'data/SEP15-OCT01TRXDATA.xlsx'


trascactions_to_get_reward_sheet_name = 'JULY1-15' 
transactions_sheet_name = 'JULY1-15'
relevancy_sheet_name = 'sheet1'



In [10]:
import pandas as pd

def parse_input(transactions_file_name, transactions_sheet_name, 
                  relevancy_file_name, relevancy_sheet_name, 
                  transactions_to_get_reward, 
                  trascactions_to_get_reward_sheet_name):
    xls = pd.ExcelFile(transaction_file_name)
    df1 = xls.parse(transactions_sheet_name)

    xls = pd.ExcelFile(relevancy_file_name)
    df2 = xls.parse(relevancy_sheet_name)

    xls = pd.ExcelFile(transactions_to_get_reward)
    df3 = xls.parse(trascactions_to_get_reward_sheet_name)

    return df1, df2, df3

print('Load data to memory')
transactions, personal_info, data_to_get_reward = parse_input(transactions_file_name, transactions_sheet_name, 
                                                              relevancy_file_name, relevancy_sheet_name, 
                                                              transactions_to_get_reward, 
                                                              trascactions_to_get_reward_sheet_name)

Load data to memory


In [12]:
from math import log

def update_rs(top_50, transactions, coof):
    # models for class 1 - 4 rs^2 * alpha + rs * alpha 2
    # increase top 50 relevancy if bought

    # increase class  1 RESPOND SHOPED
    # increase class 2 SHOPED
    # increase class 3 RESPOND
    # increase class 4 opened
    # decrease top 10 relevancy if not exist

    token = 0

    new_rs = []

    for index, row in top_50.iterrows():
        token += 1
        row = dict(row)

        result = transactions[transactions.FGC == row['fgc']]

        if result.shape[0] != 0:
            try:
                index1, row1 = next(result.iterrows())
                row1 = dict(row1)

                if row1['SHOPPED'] == 1 and row1['RESPOND'] == 1:
                    value = min(float(row['rs'] + coof[3] * abs(log(row['rs']))), 1)
                    new_rs.append(value)

                elif row1['SHOPPED'] == 1:
                    value = min(float(row['rs'] + coof[2] * abs(log(row['rs']))), 1)
                    new_rs.append(value)

                elif row1['RESPOND'] == 1:
                    value = min(float(row['rs'] + coof[1] * abs(log(row['rs']))), 1)
                    new_rs.append(value)
                elif row1['OPENED_email'] == 1:
                    value = min(float(row['rs'] + coof[0] * abs(log(row['rs']))), 1)
                    new_rs.append(value)
                else:
                    new_rs.append(row['rs'])
            except AttributeError:
                print (result)

        else:
            if token <= 10:
                new_rs.append(row['rs'] - (row['rs'] * coof[4]))
            else:
                new_rs.append(row['rs'])

    return new_rs






def get_reward(changed_rs, person_top_50, person_transactions_by_the_next_period):
    prev_rs = person_top_50['rs']
    prev_rs = list(prev_rs)

    in_top_10_current = 0
    out_of_top_10_current = 0

    in_top_10_changed = 0
    out_of_top_10_changed = 0

    sorter_changed_rs = sorted(changed_rs, reverse=True)
    token = -1

    for index, row in person_top_50.iterrows():

        token += 1
        row = dict(row)
        result = person_transactions_by_the_next_period[person_transactions_by_the_next_period.FGC == row['fgc']]

        if result.shape[0] > 1:
            result = result.iloc[0]

        if result.shape[0] != 0:
            if token <= 9:
                in_top_10_current += 1
            else:
                out_of_top_10_current += 1

            if changed_rs[token] > sorter_changed_rs[10]:
                in_top_10_changed += 1
            else:
                out_of_top_10_changed += 1

    return in_top_10_changed, out_of_top_10_changed, in_top_10_current, out_of_top_10_current


def get_whole_reward(coof, personal_info, transactions, data_to_get_reward):
    token = 0
    reward = 0
    person_id = set(list(personal_info.collector_key))

    whole_top_10_changed = 0
    whole_out_of_top_10_changed = 0

    whole_top_10_current = 0
    whole_out_of_top_10_current = 0

    for id in person_id:
        token += 1
        if token % 100 == 0:
            print(coof , '\n')
            print(whole_top_10_changed - whole_top_10_current)


        person_transactions = transactions.loc[transactions.COLLECTOR_KEY == id]
        person_transactions_by_the_next_period = data_to_get_reward.loc[data_to_get_reward.COLLECTOR_KEY == id]

        person_top_50 = personal_info.loc[personal_info.collector_key == id]

        if person_transactions.shape[0] != 0:
            new_dataset = person_transactions.loc[:, ['FGC', 'OPENED_email', 'SHOPPED', 'RESPOND']]
            rs = update_rs(person_top_50, new_dataset, coof)

            personal_info.loc[personal_info.collector_key == id, 'rs'] = rs

            top_10_changed, out_of_top_10_changed, top_10_current, \
            out_of_top_10_current = get_reward(rs, person_top_50, person_transactions_by_the_next_period)

            whole_top_10_current += top_10_current
            whole_out_of_top_10_changed += out_of_top_10_changed
            whole_top_10_changed += top_10_changed
            whole_out_of_top_10_current += out_of_top_10_current


    return whole_top_10_changed - whole_top_10_current



coof = [0, 0, 0.3, 0.3, 0.05]
max_reward = 0

get_whole_reward(coof, personal_info, transactions, data_to_get_reward)



([0, 0, 0.3, 0.3, 0.05], '\n')
0
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
2
([0, 0, 0.3, 0.3, 0.05], '\n')
2
([0, 0, 0.3, 0.3, 0.05], '\n')
2
([0, 0, 0.3, 0.3, 0.05], '\n')
2
([0, 0, 0.3, 0.3, 0.05], '\n')
2
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
1
([0, 0, 0.3, 0.3, 0.05], '\n')
0
([0, 0, 0.3, 0.3, 0.05], '\n')
0
([0, 0, 0.3, 0.3, 0.05], '\n')
0


KeyboardInterrupt: 